### Notes
- I had tried building a new dataset in "2020-06-24" but kernel kept on exploding. 
- So here, first I tried iterating through chunks of preprocessing at a time. That still exploded the kernel
- But then [midway](#2020-06-27) I tried out a really cool numpy feature where you can save an array to a file in append mode. 
- And the preprocessing step (using encoders to build transformed data and saving that to disk) , on the `843,416` rows here, took about `39` seconds. 

In [1]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1
import fresh.preproc.v2 as pv2

In [2]:
datadir = '/opt/data'
localdir = '/opt/program'


tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv'
                     )#.sample(frac=0.017, random_state=42)
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)


In [3]:

X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)
# 
# future thinking here...
# ..disk approach => return the location of X, y on disk instead 

In [4]:
workdir = fu.make_work_dir(); print(workdir)
fu.log(workdir, 'Starting')

/opt/program/artifacts/2020-06-26T142154Z


In [7]:
# place a small section of the pre-processed data into a target file..
x_outfile = f'{workdir}/x_transformed.csv'
y_outfile = f'{workdir}/y_enc.csv'

X_transformed, y_enc, proc_dict = pv2.preprocess(
        X[:1000], y[:1000], neighborhoods, labeled=True)

100%|██████████| 10/10 [00:00<00:00, 359.14it/s]


In [11]:
np.savetxt(x_outfile, X_transformed, delimiter=',')

In [16]:
(X_transformed.shape, y_enc.shape, 
 np.hstack((np.resize(y_enc, (1000, 1)), X_transformed)).shape)

((1000, 85), (1000,), (1000, 86))

In [18]:
both_outfile = f'{workdir}/minidata.csv'
yx_data = np.hstack((np.resize(y_enc, (1000, 1)), X_transformed))
np.savetxt(both_outfile, yx_data, delimiter=',', fmt='%u')

### 2020-06-27

#### trying a looped numpy append technique
* Read [here](https://stackoverflow.com/questions/27786868/python3-numpy-appending-to-a-file-using-numpy-savetxt#27980725) that you can pass a file description to `np.savetxt` to accomplish appending. 

In [28]:
# just writing same data a few times first..
both_outfile = f'{workdir}/minidata.csv'
yx_data = np.hstack((np.resize(y_enc, (1000, 1)), X_transformed))
with open(both_outfile, 'ab') as fd:
    np.savetxt(fd, yx_data, delimiter=',', fmt='%u')
    np.savetxt(fd, yx_data, delimiter=',', fmt='%u')
    

In [29]:
array = np.loadtxt(both_outfile, delimiter=',')
array.shape

(3000, 86)

In [31]:
# nice!!! that worked .
dataset_name = None#'train'
outfile = f'{workdir}/{dataset_name or "data"}.csv'
outfile

'/opt/program/artifacts/2020-06-26T142154Z/data.csv'

In [47]:
reload(pv2)

<module 'fresh.preproc.v2' from '/opt/program/fresh/preproc/v2.py'>

In [48]:
%%time
with ipdb.launch_ipdb_on_exception():
    proc_bundle, outfile = pv2.preprocess(
        X, y, neighborhoods, workdir=workdir,
        dataset_name='train')

100%|██████████| 11/11 [00:38<00:00,  3.51s/it]

CPU times: user 7.39 s, sys: 3.74 s, total: 11.1 s
Wall time: 39.6 s


Ok cool that appears to have worked 

In [50]:
proc_bundle, outfile

({'enc': OneHotEncoder(categories=[['Alphabet City', 'Battery Park City',
                             'Bedford-Stuyvesant', 'Bloomingdale', 'Boerum Hill',
                             'Bowery', 'Broadway Triangle', 'Brooklyn Heights',
                             'Brooklyn Navy Yard', 'Carnegie Hill',
                             'Carroll Gardens', 'Central Park', 'Chelsea',
                             'Chinatown', 'Civic Center', 'Clinton Hill',
                             'Cobble Hill', 'Columbia Street Waterfront District',
                             'Downtown Brooklyn', 'Dumbo', 'East Harlem',
                             'East Village', 'East Williamsburg',
                             'Financial District', 'Flatiron District',
                             'Fort Greene', 'Fulton Ferry District',
                             'Garment District', 'Governors Island', 'Gowanus', ...],
                            [0, 1, 2], [0, 1, 2, 3, 4]],
                drop=None, dtype=<class 

In [51]:
!ls -lh '/opt/program/artifacts/2020-06-26T142154Z/train.csv'

-rw-r--r-- 1 root root 139M Jun 27 18:38 /opt/program/artifacts/2020-06-26T142154Z/train.csv


In [52]:
import sklearn.datasets as sd

In [54]:
yx_data.shape

(1000, 86)

In [57]:
both_outfile = f'{workdir}/minidata.libsvm'
with open(both_outfile, 'ab') as fd:
    sd.dump_svmlight_file(yx_data[:100, 1:], yx_data[:100, 0], fd)
    sd.dump_svmlight_file(yx_data[100:200, 1:], yx_data[100:200, 0], fd)
    sd.dump_svmlight_file(yx_data[200:300, 1:], yx_data[200:300, 0], fd)
both_outfile

'/opt/program/artifacts/2020-06-26T142154Z/minidata.libsvm'

In [67]:
%%time
reload(pv2)
X_train, X_test, y_train, y_test = train_test_split(X, y)
proc_bundle, outfile = pv2.preprocess(
        X_train, y_train, neighborhoods, workdir=workdir,
        dataset_name='train')
outfile

100%|██████████| 11/11 [00:46<00:00,  4.19s/it]

CPU times: user 21.5 s, sys: 370 ms, total: 21.9 s
Wall time: 47.3 s


'/opt/program/artifacts/2020-06-26T142154Z/train.libsvm'

In [20]:
dmatrix = xgb.DMatrix(
    f'{both_outfile}?format=csv&label_column=0&delimiter=,')


[15:28:46] 1000x85 matrix with 85000 entries loaded from /opt/program/artifacts/2020-06-26T142154Z/minidata.csv?format=csv&label_column=0&delimiter=,


In [21]:
xgb_model = xgb.XGBClassifier(objective='multi:softprob')
xgb_model.fit(dmatrix,  verbose=True)

TypeError: fit() missing 1 required positional argument: 'y'

##### thoughts on params to mess with..
num_round , make sure at least 100
gamma, 0
max_delta_step , 1
n_estimators , >100..
min_child_weight, 30
max_depth , 3,4,5,6...
colsample_bytree, 0.4..1.0 
subsample, 0.5..1.0


#### references
- [this](https://github.com/aws/sagemaker-xgboost-container/blob/master/src/sagemaker_xgboost_container/data_utils.py) is a handy reference around xgb utils

In [35]:
os.getpid()

1342